## Import packages

In [1]:
import numpy as np 
import csv 
import glob
import pandas as pd
import os
import sys
from tqdm import tqdm

# Get the path to the parent directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the Python path
sys.path.append(parent_dir)

# Import utils folder
from utils.twostep_support import *    
from models import *

## Set number of parameter sets (number of synthetic participants) and select model class

In [7]:
model = "AI" # RL or AI
learning = "PSM" # RL, PSM
mtype = 0
dataset = 'magic_carpet_2023'
n_param_sets = 1 #Number of parameter sets or 'synthetic participants'

## Generate parameter sets (synthetic participants)

In [8]:
# Different model setups
if model == "RL":
    p_names = ["Synthetic_lr1", 
               "Synthetic_lr2", 
               "Synthetic_lam", 
               "Synthetic_b1", 
               "Synthetic_b2", 
               "Synthetic_p", 
               "Synthetic_w"]
    lower_bounds = np.array([0, 0, 0, 0, 0, -1, 0])
    upper_bounds = np.array([1, 1, 1, 20, 20, 1, 1])
elif model == "AI":
    if mtype == 0:
        p_names = ["Synthetic_lr",
                   "Synthetic_vunsamp", 
                   "Synthetic_vsamp", 
                   "Synthetic_vps", 
                   "Synthetic_gamma1", 
                   "Synthetic_gamma2", 
                   "Synthetic_lam", 
                   "Synthetic_kappa_a", 
                   "Synthetic_prior_r"]
        lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0.2])
        upper_bounds = np.array([4, 0.9, 0.9, 0.9, 30, 30, 10, 5, 0.8])
    elif mtype == 1:
        p_names = ["Synthetic_lr",
                   "Synthetic_vunsamp", 
                   "Synthetic_vps", 
                   "Synthetic_gamma1", 
                   "Synthetic_gamma2", 
                   "Synthetic_lam", 
                   "Synthetic_kappa_a", 
                   "Synthetic_prior_r"]
        lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0, 0.2])
        upper_bounds = np.array([4, 0.9, 0.9, 30, 30, 10, 5, 0.8])
    elif mtype == 2:
        p_names = ["Synthetic_lr", 
                   "Synthetic_vsamp", 
                   "Synthetic_vps", 
                   "Synthetic_gamma1", 
                   "Synthetic_gamma2", 
                   "Synthetic_lam", 
                   "Synthetic_kappa_a", 
                   "Synthetic_prior_r"]
        lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0, 0.2])
        upper_bounds = np.array([4, 0.9, 0.9, 30, 30, 10, 5, 0.8])
    elif mtype == 3:
        p_names = ["Synthetic_lr",
                   "Synthetic_vunsamp", 
                   "Synthetic_vsamp", 
                   "Synthetic_gamma1", 
                   "Synthetic_gamma2", 
                   "Synthetic_lam", 
                   "Synthetic_kappa_a", 
                   "Synthetic_prior_r"]
        lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0, 0.2])
        upper_bounds = np.array([4, 0.9, 0.9, 30, 30, 10, 5, 0.8]) 
        
n_free_params = len(p_names)
        
param_sets = np.zeros((n_param_sets, n_free_params+1))

p_names.insert(0, "Synthetic_participant_ID")


df_param_sets = pd.DataFrame(param_sets, columns = p_names)
        
        
for i in range(n_param_sets):
    
    df_param_sets.loc[i,'Synthetic_participant_ID'] = i + 1
    
    for j in range(n_free_params):
        
        df_param_sets.loc[i,p_names[j+1]] = np.random.uniform(lower_bounds[j],upper_bounds[j])

In [9]:
df_param_sets.head()

,Synthetic_participant_ID,Synthetic_lr,Synthetic_vunsamp,Synthetic_vsamp,Synthetic_vps,Synthetic_gamma1,Synthetic_gamma2,Synthetic_lam,Synthetic_kappa_a,Synthetic_prior_r
0,1.0,1.854228,0.301276,0.478963,0.256272,3.667837,20.04867,6.813598,4.335696,0.279429


## Save parameter sets

In [10]:
#directory = 'param_recovery_data/' + dataset + '/model_'+ model+'_'+str(mtype)

# Create the directory if it doesn't exist
#os.makedirs(directory, exist_ok=True)

# File path within the new directory
#file_path = os.path.join(directory, "param_recovery_synthetic_params_M"+model+str(mtype)+".csv")

# Save the DataFrame to a CSV file
#df_param_sets.to_csv(file_path, index=False)

## Simulate rounds for each parameter set (i.e., generate synthetic data)

In [12]:
list_dict_simulation_results = []


task_dict = {
    "type": "drift", # How outcome probabilities vary
    "T": 150, # number of trials
    "x": False, # Whether transition probabilities are resampled
    "r": True, # Whether outcome probabilities are resampled
    "delta": 0.025, # The volatility of task statistics (variance of Gaussian for drift-version aka, diffusion rate)
    "bounds": [0.25,0.75]# List of 2 floats; lower and upper bounds of (final-stage) outcome probabilities
}

model_dict = {
    "act": "AI",
    "learn": "PSM",
    "learn_transitions": True
}

for i in tqdm(range(n_param_sets)):

    # Get the parameter sets of this participant
    df_param_set = df_param_sets[df_param_sets['Synthetic_participant_ID'] == i+1]
    
    if model == "RL":
        
        model_dict['lr1'] = df_param_set['Synthetic_lr1'].iloc[0]
        model_dict['lr2'] = df_param_set['Synthetic_lr2'].iloc[0]
        model_dict['lam'] = df_param_set['Synthetic_lam'].iloc[0]
        model_dict['b1'] = df_param_set['Synthetic_b1'].iloc[0]
        model_dict['b2'] = df_param_set['Synthetic_b2'].iloc[0]
        model_dict['p'] = df_param_set['Synthetic_p'].iloc[0]
        model_dict['w'] = df_param_set['Synthetic_w'].iloc[0]
          
    elif model == "AI":
        if mtype == 0:
            model_dict['lr'] = df_param_set["Synthetic_lr"].iloc[0]
            model_dict['vunsamp'] = df_param_set["Synthetic_vunsamp"].iloc[0] 
            model_dict['vsamp'] = df_param_set["Synthetic_vsamp"].iloc[0] 
            model_dict['vps'] = df_param_set["Synthetic_vps"].iloc[0]
            model_dict['gamma1'] = df_param_set["Synthetic_gamma1"].iloc[0] 
            model_dict['gamma2'] = df_param_set["Synthetic_gamma2"].iloc[0] 
            model_dict['lam'] = df_param_set["Synthetic_lam"].iloc[0]
            model_dict['kappa_a'] = df_param_set["Synthetic_kappa_a"].iloc[0] 
            model_dict['prior_r'] = df_param_set["Synthetic_prior_r"].iloc[0]

        elif mtype == 1:
            model_dict["lr"] = df_param_set["Synthetic_lr"].iloc[0]
            model_dict["vunsamp"] = df_param_set["Synthetic_vunsamp"].iloc[0]
            model_dict["vps"] = df_param_set["Synthetic_vps"].iloc[0]
            model_dict["gamma1"] = df_param_set["Synthetic_gamma1"].iloc[0] 
            model_dict["gamma2"] = df_param_set["Synthetic_gamma2"].iloc[0]
            model_dict["lam"] = df_param_set["Synthetic_lam"].iloc[0]
            model_dict["kappa_a"] = df_param_set["Synthetic_kappa_a"].iloc[0] 
            model_dict["prior_r"] = df_param_set["Synthetic_prior_r"].iloc[0]
   
        elif mtype == 2:
            model_dict["lr"] = df_param_set["Synthetic_lr"].iloc[0] 
            model_dict["vsamp"] = df_param_set["Synthetic_vsamp"].iloc[0]
            model_dict["vps"] = df_param_set["Synthetic_vps"].iloc[0]
            model_dict["gamma1"] = df_param_set["Synthetic_gamma1"].iloc[0] 
            model_dict["gamma2"] = df_param_set["Synthetic_gamma2"].iloc[0] 
            model_dict["lam"] = df_param_set["Synthetic_lam"].iloc[0] 
            model_dict["kappa_a"] = df_param_set["Synthetic_kappa_a"].iloc[0] 
            model_dict["prior_r"] = df_param_set["Synthetic_prior_r"].iloc[0]

        elif mtype == 3:
            model_dict["lr"] = df_param_set["Synthetic_lr"].iloc[0]
            model_dict["vunsamp"] = df_param_set["Synthetic_vunsamp"].iloc[0] 
            model_dict["vsamp"] = df_param_set["Synthetic_vsamp"].iloc[0]
            model_dict["gamma1"] = df_param_set["Synthetic_gamma1"].iloc[0] 
            model_dict["gamma2"] = df_param_set["Synthetic_gamma2"].iloc[0]
            model_dict["lam"] = df_param_set["Synthetic_lam"].iloc[0] 
            model_dict["kappa_a"] = df_param_set["Synthetic_kappa_a"].iloc[0] 
            model_dict["prior_r"] = df_param_set["Synthetic_prior_r"].iloc[0]
            
    agent = learn_and_act(task = task_dict, model = model_dict)

    actions, observations, pi, p_trans, p_r, Gs = agent.perform_task()
    
    simulation_results_formatted = {"Synthetic_participant_ID": i+1,
                                    "choice1": actions[:,0].tolist(),
                                    "choice2": actions[:,1].tolist(),
                                    "final_state": observations[:,0].tolist(),
                                    "reward": observations[:,1].tolist()
                                   }

    list_dict_simulation_results.append(simulation_results_formatted)

df_simulation_results = pd.DataFrame(list_dict_simulation_results)

100%|██████████| 1/1 [00:00<00:00,  4.64it/s]


## Save synthetic data

In [18]:
file_path = os.path.join(directory, "param_recovery_synthetic_data_M" + model + str(mtype) + ".csv")
df_simulation_results.to_csv(file_path, index=False)